# Learning: Building a Custom Sensor

This notebook guides you through building your own sensor. **You'll need to fill in the missing code!**

## What You'll Learn

1. Understanding the sensor interface (`BaseSensor`)
2. Implementing a simple range sensor
3. Implementing a camera sensor (simulated)
4. Testing your implementation

## Instructions

- Read each section carefully
- Look for `# TODO:` comments - these indicate where you need to write code
- Fill in the `...` placeholders
- Run cells as you complete them to test your work

## Prerequisites

- Understanding of sensors (see [Building Custom Sensors](../building/building_custom_sensors.ipynb))
- Understanding of perception data (`PerceptionPoints`)
- Basic numpy knowledge

In [ ]:
import sys
sys.path.insert(0, '../../src')

import numpy as np
import matplotlib.pyplot as plt
from simple_autonomous_car import (
    Track,
    Car,
    CarState,
    GroundTruthMap,
    PerceivedMap,
    BaseSensor,
    PerceptionPoints,
)

## Step 1: Understanding the BaseSensor Interface

All sensors must inherit from `BaseSensor` and implement the `sense()` method.

In [ ]:
from simple_autonomous_car.sensors.base_sensor import BaseSensor
import inspect

print("BaseSensor interface:")
print("  sense(car_state, environment_data) -> PerceptionPoints")
print("  Returns: PerceptionPoints with points and frame")
print("\nKey points:")
print("  - Must return PerceptionPoints object")
print("  - Points can be in 'ego' or 'global' frame")
print("  - environment_data typically contains 'ground_truth_map'")
print("  - Can have pose_ego attribute for sensor position on car")

## Step 2: Build a Simple Range Sensor

Let's build a simple forward-facing range sensor.

## Step 3: Test Your Sensor

Once you've filled in all the TODOs, test your sensor!

In [ ]:
# Setup test
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)

start_point, start_heading = track.get_point_at_distance(0.0)
car_state = CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0)

# TODO: Create your sensor
my_sensor = SimpleRangeSensor(
    ground_truth_map,
    max_range=25.0,
    fov_angle=np.pi / 3,  # 60 degrees
    num_rays=15,
)

# TODO: Sense obstacles
perception = my_sensor.sense(car_state, environment_data={"ground_truth_map": ground_truth_map})

print(f"✓ Sensor detected {len(perception.points)} points")
if len(perception.points) > 0:
    print(f"  First point: ({perception.points[0][0]:.2f}, {perception.points[0][1]:.2f})")

In [ ]:
# Visualize sensor data
from simple_autonomous_car.visualization import plot_perception, plot_car

fig, ax = plt.subplots(figsize=(12, 10))

track.visualize(ax=ax, frame="global")
plot_perception(perception, car_state, ax=ax, frame="global", color="red", label="Your Sensor")

car = Car(initial_state=car_state)
plot_car(car, ax=ax, show_heading=True)

# Show sensor FOV (simplified visualization)
sensor_pos = car_state.position()
sensor_heading = car_state.heading
fov = np.pi / 3
for angle in [sensor_heading - fov/2, sensor_heading + fov/2]:
    ray_end = sensor_pos + 25.0 * np.array([np.cos(angle), np.sin(angle)])
    ax.plot([sensor_pos[0], ray_end[0]], [sensor_pos[1], ray_end[1]], "g--", alpha=0.3)

ax.set_title("Your Simple Range Sensor")
ax.legend()
plt.show()

print("✓ Visualization complete!")

## Summary

Congratulations! You've built your own sensor! 🎉

### What You Learned

1. ✅ **Sensor interface**: Understanding `BaseSensor` and `sense()` method
2. ✅ **Range sensing**: Ray casting for obstacle detection
3. ✅ **Frame transformations**: Converting between ego and global frames

### Key Concepts

- **Field of View (FOV)**: Angular range of sensor
- **Max range**: Maximum detection distance
- **PerceptionPoints**: Standard format for sensor data
- **Frame**: Ego (car frame) vs Global (world frame)

### Next Steps

- Implement proper ray casting with line intersection
- Add noise to sensor measurements
- Build a camera or radar sensor
- Add sensor fusion (combine multiple sensors)

In [ ]:
class SimpleRangeSensor(BaseSensor):
    """
    A simple forward-facing range sensor.
    
    TODO: Fill in the implementation!
    """
    
    def __init__(
        self,
        ground_truth_map,
        max_range: float = 20.0,
        fov_angle: float = np.pi / 4,  # 45 degrees
        num_rays: int = 10,
        name: str = "simple_range",
    ):
        """
        Initialize the sensor.
        
        TODO:
        1. Call super().__init__(name=name, max_range=max_range)
        2. Store ground_truth_map as instance variable
        3. Store fov_angle, num_rays
        """
        # TODO: Initialize
        super().__init__(name=name, max_range=max_range)
        # Store ground_truth_map for obstacle detection
        self.ground_truth_map = ground_truth_map
        self.fov_angle = ...
        self.num_rays = ...
    
    def sense(self, car_state: CarState, environment_data: dict) -> PerceptionPoints:
        """
        Sense obstacles in a forward cone.
        
        TODO:
        1. Get sensor pose in global frame
        2. Generate rays in field of view
        3. Return PerceptionPoints with detected points
        """
        # TODO: Get sensor pose
        if hasattr(self, 'pose_ego'):
            sensor_pos_ego = self.pose_ego[:2]
            sensor_heading_ego = self.pose_ego[2]
            sensor_pos = car_state.transform_to_world_frame(sensor_pos_ego)
            sensor_heading = car_state.heading + sensor_heading_ego
        else:
            sensor_pos = ...
            sensor_heading = ...
        
        # TODO: Get track from ground truth map
        track = ...
        
        # TODO: Generate rays and detect obstacles
        detected_points = []
        
        for i in range(self.num_rays):
            # Calculate ray angle
            angle_offset = ...
            ray_angle = ...
            
            # Calculate ray end point
            ray_end = ...
            
            # Check if near track boundary
            min_dist = min(
                np.min(np.linalg.norm(track.inner_bound - ray_end, axis=1)),
                np.min(np.linalg.norm(track.outer_bound - ray_end, axis=1))
            )
            
            if min_dist < 1.0:  # Within 1m of boundary
                detected_points.append(ray_end)
        
        # TODO: Return PerceptionPoints
        if len(detected_points) == 0:
            return PerceptionPoints(points=np.array([]).reshape(0, 2), frame="global")
        
        return PerceptionPoints(points=..., frame="global")

print("✓ Simple range sensor structure created!")
print("  Now fill in the TODOs!")